In [13]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import FinanceDataReader as fdr
from tensorflow.keras.layers import Dense, LSTM, Conv1D, Lambda
from tensorflow.keras.losses import Huber


In [8]:
def load_and_preprocess_data(STOCK_CODE):
    # 데이터 불러오기
    df = fdr.DataReader(STOCK_CODE)    
    df['Year'] = df.index.year
    df['Month'] = df.index.month
    df['Day'] = df.index.day
# '년', '월', '일' 필드를 'date' 필드로 변경
    df['date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
    # 데이터 전처리
    df['date'] = pd.to_datetime(df['date'])
    df.sort_values('date', inplace=True)
    df.set_index('date', inplace=True)
    
    # 데이터 스케일링
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(df['Close'].values.reshape(-1, 1))
    
    return scaled_data, scaler


In [11]:
def create_and_train_lstm_model(data, lookback):
    # 입력 데이터 생성
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:(i + lookback), 0])
        y.append(data[i + lookback, 0])
    X, y = np.array(X), np.array(y)
    
    # 데이터 분할 (학습 데이터와 검증 데이터)
    train_size = int(len(X) * 0.8)
    X_train, X_val = X[:train_size], X[train_size:]
    y_train, y_val = y[:train_size], y[train_size:]
    WINDOW_SIZE=20
    BATCH_SIZE=32
    # trian_data는 학습용 데이터셋, test_data는 검증용 데이터셋 입니다.
    train_data = windowed_dataset(y_train, WINDOW_SIZE, BATCH_SIZE, True)
    test_data = windowed_dataset(y_test, WINDOW_SIZE, BATCH_SIZE, False)
    for data in train_data.take(1):
        print(f'데이터셋(X) 구성(batch_size, window_size, feature갯수): {data[0].shape}')
        print(f'데이터셋(Y) 구성(batch_size, window_size, feature갯수): {data[1].shape}')
    # LSTM 모델 생성
    # model = Sequential()
    # model.add(LSTM(50, input_shape=(lookback, 1)))
    # model.add(Dense(1))
    # model.compile(loss='mean_squared_error', optimizer='adam')
    model = Sequential([
    # 1차원 feature map 생성
    Conv1D(filters=32, kernel_size=5,
           padding="causal",
           activation="relu",
           input_shape=[WINDOW_SIZE, 1]),
    # LSTM
    LSTM(16, activation='tanh'),
    Dense(16, activation="relu"),
    Dense(1),
])

    # 모델 학습
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=32)
    
    return model


In [4]:
def predict_future_stock_price(model, data, scaler, lookback, prediction_days=7):
    last_data = data[-lookback:]
    prediction = []

    for _ in range(prediction_days):
        # 데이터 스케일링
        scaled_data = scaler.transform(last_data.reshape(-1, 1))

        # 입력 데이터 생성
        X = np.array([scaled_data])

        # 주가 예측
        predicted_price = model.predict(X)

        # 예측 결과 역스케일링
        predicted_price = scaler.inverse_transform(predicted_price)

        # 예측값 저장
        prediction.append(predicted_price[0][0])

        # 다음 날 예측을 위해 데이터 업데이트
        last_data = np.append(last_data[1:], predicted_price[0])

    return prediction


In [14]:
# 예측을 위한 데이터 전처리
# filename = 'stock_data.csv'  # 주식 데이터 파일 경로
# 삼성전자 주식코드: 005930
STOCK_CODE = '005930'
lookback = 60  # LSTM 입력 시퀀스 길이

data, scaler = load_and_preprocess_data(STOCK_CODE)
# 학습된 모델 로드
model = create_and_train_lstm_model(data, lookback)
# 7일간의 주가 예측
prediction = predict_future_stock_price(model, data, scaler, lookback, prediction_days=7)

print(prediction)


NameError: name 'WINDOW_SIZE' is not defined